In [ ]:
# !pip install faiss-cpu #if you are using 'chroma', then this is not required
!pip install pypdf
!pip install langchain_community
!pip install langchain_huggingface
!pip install chromadb

In [3]:
from langchain_community.document_loaders import PyPDFLoader

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [5]:
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings

In [6]:
from transformers import AutoModelForCausalLM,AutoTokenizer,pipeline,BitsAndBytesConfig
import torch

In [7]:
# model_id = 'models--meta-llama--Llama-3.2-3B-Instruct\\snapshots\\0cb88a4f764b7a12671c53f0838cd831a0843b95'
model_id = 'Qwen/Qwen3-4B-Instruct-2507'
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True, #4-bit quantization
# )

tokenizer = AutoTokenizer.from_pretrained(model_id,device_map='cuda')
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,#single quant
    device_map = 'cuda',
)
# tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [32]:
pipe=pipeline("text-generation",model=model,
              tokenizer=tokenizer,max_new_tokens=512,return_full_text = False,
              do_sample=True, temperature=0.2,
              repetition_penalty=1.03)
llm=HuggingFacePipeline(pipeline=pipe)

Device set to use cuda


In [9]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [10]:
# loader = PyPDFLoader("mukti_legal_land_verdict.pdf")
# loader = PyPDFLoader("Nestle_Annual-Report-2023-24_abstract.pdf")
loader = PyPDFLoader("bns.pdf")
#pages = loader.load_and_split() #page-wise

In [11]:
# pages
loader

In [12]:
model_name = 'sentence-transformers/all-mpnet-base-v2'
model_kwargs = {"trust_remote_code":True}#"device":"cuda"
embeddings = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs,show_progress=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
docs = loader.load()

In [14]:
len(docs)

102

In [15]:
import re
len(re.findall('\w+',docs[0].page_content))

256

In [16]:
# embeddings = ""
print(docs[0].page_content)

THE BHARA TIYA NY AYA SANHITA, 2023
NO. 45 OF 2023
[25th December ,2023.]
An Act to consolidate and amend the provisions relating to offences and for
matters connected therewithor incidental thereto.
BE it enacted by Parliament in the Seventy-fourth Year of the Republic of India as
follows:––
CHAPTERI
PRELIMINARY
1.(1) This Act may be called the Bharatiya Nyaya Sanhita, 2023.
(2) It shall come into force on such date as the Central Government may , bynotification
in the Official Gazette, appoint, and different dates maybe appointed for different provisions
of this Sanhita.
Short title,
commencement
and
application.
vlk/kkj.k
EXTRAORDINARY
Hkkx II — [k.M 1
PART II — Section 1
izkf/kdkj ls izdkf'kr
PUBLISHED BY AUTHORITY
lañ 53] ubZ fnYyh] lkseokj] fnlEcj 25] 2023@ ikS"k 4] 1945 ¼'kd½
No. 53] NEW DELHI, MONDAY, DECEMBER 25, 2023/PAUSHA 4, 1945 (SAKA)
bl Hkkx esa fHkUu i`"B la[;k nh tkrh gS ftlls fd ;g vyx ladyu ds :i esa j[kk tk ldsA
Separate paging is given to this Part in order that it

In [ ]:
# text_splits = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
# all_splits = text_splits.split_documents(docs)

In [17]:
# all_splits[0]

In [18]:
## we will store each page in chroma db
li_all_txt = []
for txt in docs:
    li_all_txt.append(txt.page_content)
li_all_txt[:2]

['THE BHARA TIYA NY AYA SANHITA, 2023\nNO. 45 OF 2023\n[25th December ,2023.]\nAn Act to consolidate and amend the provisions relating to offences and for\nmatters connected therewithor incidental thereto.\nBE it enacted by Parliament in the Seventy-fourth Year of the Republic of India as\nfollows:––\nCHAPTERI\nPRELIMINARY\n1.(1) This Act may be called the Bharatiya Nyaya Sanhita, 2023.\n(2) It shall come into force on such date as the Central Government may , bynotification\nin the Official Gazette, appoint, and different dates maybe appointed for different provisions\nof this Sanhita.\nShort title,\ncommencement\nand\napplication.\nvlk/kkj.k\nEXTRAORDINARY\nHkkx II — [k.M 1\nPART II — Section 1\nizkf/kdkj ls izdkf\'kr\nPUBLISHED BY AUTHORITY\nlañ 53] ubZ fnYyh] lkseokj] fnlEcj 25] 2023@ ikS"k 4] 1945 ¼\'kd½\nNo. 53] NEW DELHI, MONDAY, DECEMBER 25, 2023/PAUSHA 4, 1945 (SAKA)\nbl Hkkx esa fHkUu i`"B la[;k nh tkrh gS ftlls fd ;g vyx ladyu ds :i esa j[kk tk ldsA\nSeparate paging is given

In [19]:
# vectordb = Chroma.from_documents(documents=li_all_txt,embedding=embeddings)#persist_directory = "chroma_db"
vectordb = Chroma.from_texts(texts=li_all_txt[1:],embedding=embeddings)#persist_directory = "chroma_db"

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
retriever = vectordb.as_retriever(search_kwargs={'k':2},search_type="mmr")#'fetch_k':50

In [26]:
# query = "what is total equities and liabilities for the year 31 March 2024?"
query = "cruelty against a married women by husband"
retriever.invoke(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[]

In [27]:
vectordb.similarity_search_with_score(query,k=3)
# retriever = vectordb.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5,"k":2}
# )
# retriever.invoke(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(Document(metadata={}, page_content='from such person for the space of seven years, and shall not have been heard of by such\nperson as being alive within that time provided the person contracting such subsequent\nmarriage shall, before such marriage takes place, inform the person with whom such marriage\nis contracted of the real state of facts so far as the same are within his or her knowledge.\n(2) Whoever commits the offence under sub-section (1) having concealed from the\nperson with whom the subsequent marriage is contracted, the fact of the former marriage,\nshall be punished with imprisonment of either description for a term which may extend to ten\nyears, and shall also be liable to fine.\n83.Whoever, dishonestly or with a fraudulent intention, goes through the ceremony\nof being married, knowing that he is not thereby lawfully married, shall be punished with\nimprisonment of either description for a term which may extend to seven years, and shall also\nbe liable to fine.\n84

In [ ]:
## Retrive data using LLMs
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    verbose=True
)
query = "what is total liabilities for the year 31 March 2024?"
output = qa.invoke(query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.


In [ ]:
print(output['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

BALANCE SHEET AS AT 31 MARCH 2024
(H in million)
NOTES As at  
31 March 2024
As at  
31 December 2022
ASSETS 
Non-current assets 
Property, Plant and Equipment 6  30,556.7  27,058.1 
Capital work-in-progress 6  17,417.1  3,583.6 
Right of Use Assets 6  4,045.8  3,378.9 
Financial Assets 
 Investments 7  4,238.9  5,601.9 
 Loans 8  80.1  90.0 
 Other financial assets 9  565.1  468.3 
Deferred tax assets (net) 10  -  256.1 
Other non-current assets 11  13,379.7  14,449.0 
 70,283.4  54,885.9 
Current assets 
Inventories 12  20,893.6  19,287.7 
Financial assets 
 Investments 7  399.8  2,173.5 
 Trade receivables 13  3,004.6  1,918.9 
 Cash and cash equivalents 14  7,588.7  9,265.5 
 Bank balances other than cash and cash equivalents 15  199.8  190.0 
 Loans 8  110.1  102.6 
 Other financial assets 9  462.7  582.6 
Other current

In [33]:
### Query expansion
prompt1 = """ you are a legal AI assistant. who helps user in query expansion. Your task is provide expanded query point wise without any additional comment.
"""
prompt2 = f""" Here is the query: {query}
"""

messages = [{"role": "system", "content": prompt1},
    {"role": "user", "content": prompt2}
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

In [35]:
result=pipe(prompt)[0]['generated_text']

In [41]:
#result = '<|im_start|>system\n you are a AI assistant. who helps user in query expansion.\n<|im_end|>\n<|im_start|>user\n Here is the query: cruelty against a married women by husband\n<|im_end|>\n<|im_start|>assistant\nGreat, I can help expand and refine your query for better searchability or clarity—especially if you\'re looking to use it for research, legal purposes, or awareness campaigns.\n\nOriginal query:  \n**"cruelty against a married women by husband"**\n\nExpanded and refined versions (depending on context):\n\n1. **General Awareness / Legal Context:**  \n   "Cruelty by husband towards married woman in Indian law"  \n   *(This is specific to India, where "cruelty" is a recognized ground under Section 498A of the Indian Penal Code.)*\n\n2. **Legal & Judicial Focus:**  \n   "Definition and legal provisions of cruelty against a married woman by her husband in India"  \n   "Cases of marital cruelty by husband leading to divorce under Hindu Marriage Act, 1955"\n\n3. **Social & Gender Issues:**  \n   "Marital abuse and domestic violence by husbands against wives in India"  \n   "Gender-based violence in marriage: husband\'s cruelty toward wife"\n\n4. **Case Law & Precedents:**  \n   "Landmark judgments on cruelty by husband in Indian family law"  \n   "Supreme Court rulings on cruelty in matrimonial disputes"\n\n5. **Prevention & Awareness:**  \n   "How to report cruelty by husband to married woman in India"  \n   "Awareness campaigns on marital cruelty and domestic violence"\n\n6. **Academic/Research Use:**  \n   "Cruelty in marriage: a study on husband-wife dynamics and legal remedies in India"  \n   "Cultural and legal factors contributing to marital cruelty in Indian society"\n\n💡 Tip: If you\'re researching for legal advice, use the Indian legal framework. If it\'s for public awareness, focus on social impact and reporting mechanisms.\n\nLet me know your intended use (e.g., legal research, article writing, advocacy), and I can tailor the expanded query even more precisely!'
# print(result)
li_res = result.replace('-','').split('\n')
li_res

[' Cruelty by a husband towards his married wife under Indian law  ',
 ' Legal provisions for cruelty in marriage as defined under Section 498A of the Indian Penal Code  ',
 ' Definition and scope of "cruelty" as applicable to married women by their husbands  ',
 ' Cases where husbands have been convicted for cruelty against wives in Indian courts  ',
 ' Remedies available to a married woman suffering cruelty at the hands of her husband  ',
 ' Difference between cruelty by husband and cruelty by inlaws  ',
 ' Role of the court in protecting married women from domestic cruelty  ',
 ' Evidence required to prove cruelty by a husband in a matrimonial dispute  ',
 ' Impact of cruelty on divorce proceedings and maintenance claims  ',
 ' Protection of married women from physical, emotional, and psychological abuse by husbands  ',
 ' Legal remedies under the Domestic Violence Act, 2005, for cruelty by a husband  ',
 ' How cruelty affects the rights of a married woman under the Hindu Marriage A

In [37]:
vectordb.similarity_search_with_score(result,k=3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(Document(metadata={}, page_content='from such person for the space of seven years, and shall not have been heard of by such\nperson as being alive within that time provided the person contracting such subsequent\nmarriage shall, before such marriage takes place, inform the person with whom such marriage\nis contracted of the real state of facts so far as the same are within his or her knowledge.\n(2) Whoever commits the offence under sub-section (1) having concealed from the\nperson with whom the subsequent marriage is contracted, the fact of the former marriage,\nshall be punished with imprisonment of either description for a term which may extend to ten\nyears, and shall also be liable to fine.\n83.Whoever, dishonestly or with a fraudulent intention, goes through the ceremony\nof being married, knowing that he is not thereby lawfully married, shall be punished with\nimprisonment of either description for a term which may extend to seven years, and shall also\nbe liable to fine.\n84

In [ ]:
### Re-ranking
# queries = [query]+li_res
queries = query + '\n'+result.replace('-','')
output = vectordb.similarity_search_with_score(queries,k=3)
#output

In [56]:
# print(output[1][0])
# unique_doc = set()
# for doc in output:
#   unique_doc.add(doc[0])
# li_doc = list(unique_doc)

#li_doc = []
seen_content = set()
for doc, score in output:
    #if doc.page_content not in seen_content:
     #   li_doc.append(doc)
    seen_content.add(doc.page_content)
li_doc = list(seen_content)

In [57]:
len(li_doc)

3

In [58]:
pairs = []
for doc in li_doc:
    pairs.append([query, doc])

In [59]:
scores = cross_encoder.predict(pairs)

In [60]:
print("Scores:")
for score in scores:
    print(score)

Scores:
2.8693671
-7.78967
-9.086568


In [62]:
import numpy as np
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
0
1
2
